## Вопрос №1

In [14]:
import pandas as pd
import numpy as np

In [15]:
# ПОСЛЕДНИЙ МЕСЯЦ
# 16.06.2019
# from datetime import datetime
# from dateutil import relativedelta
# date1 = datetime.date.today()
# orders = pd.read_csv('orders.csv')
# orderLines = pd.read_csv('orderLines.csv')
# months = []
# for i in range(len(orders)):
#     date2 = datetime.strptime(str(orders.loc[i, 'DateTime']), '%d.%m.%Y')
#     months.append(relativedelta.relativedelta(date2, date1))
# orders['months'] = months
# orders = orders.loc[orders.months <= 1]
# ind = orders.index
# orderLines = orderLines.loc[ind]

### Искусственные данные ORDERS

In [16]:
# СТРОИМ ПРИМЕРНЫЕ ORDERS
orders = pd.DataFrame()
orders['DateTime'] = pd.Series(np.arange(100)+1)
orders['CustomerID'] = pd.Series(np.round(np.random.rand(100)*20))
orders['OrderID'] = pd.Series(np.round(np.random.rand(100)*10000))
orders.head()

,DateTime,CustomerID,OrderID
0,1,14.0,1094.0
1,2,18.0,7296.0
2,3,17.0,3616.0
3,4,19.0,7325.0
4,5,5.0,3955.0


### Искусственные данные ORDERLINES

In [5]:
# СТРОИМ ПРИМЕРНЫЕ ORDERLINES
orderLines = pd.DataFrame()
orderLines['OrderID'] = orders['OrderID']
orderLines['ProductID'] = pd.Series(np.round(np.random.rand(100)*20))
orderLines['Price'] = orderLines['ProductID']*10
orderLines.head()

,OrderID,ProductID,Price
0,2394.0,16.0,160.0
1,149.0,15.0,150.0
2,284.0,17.0,170.0
3,3431.0,6.0,60.0
4,2493.0,19.0,190.0


### Отчет

In [17]:
# СТРОИМ ОТЧЕТ
report = pd.DataFrame()

report['most_popular_products_id'] = orderLines['ProductID'].value_counts().keys()
report['total_value'] = orderLines['ProductID'].value_counts().values

for i in range(len(report)):
    report.total_value.loc[i] *= orderLines.loc[orderLines.ProductID == 
                                                report.most_popular_products_id.loc[i], 'Price'].values[0]
total = []

for i in report.most_popular_products_id:
    total_p = 0
    count = 0
    
    for j in orderLines.index:
        if orderLines.loc[j, 'ProductID'] == i:
            order_id = orderLines.loc[j, 'OrderID']
            total_p += sum(orderLines.loc[orderLines.OrderID == order_id, 'Price'])
            count += 1
    total.append(total_p/count)

report['mean_value_for_order'] = total

report.head()

,most_popular_products_id,total_value,mean_value_for_order
0,9.0,810.0,90.0
1,12.0,1080.0,120.0
2,10.0,800.0,100.0
3,15.0,1050.0,150.0
4,18.0,1260.0,180.0


## Вопрос №2

1. Целевая переменная (Клиент не совершит покупку более года, начиная с текущего момента; 1 - если да, 0 - если нет)


2. Возраст, пол, семейный статус, уровень дохода (если есть), транзакции по карте (если есть), геоданные (если есть), паттерны поведения клиентов, похожих на данного.


3. Используя решающее дерево на всех собранных признаках, можно узнать какие условия являются решающими для предсказания оттока, а так же для каждого абонента предсказать целевую переменную. Это можно также сделать оценив веса логистической регрессии(признаки с большим весом имеют большую важность). Вообщем на этом шаге мы строим baseline модель, которая отделяет **важные для нас** признаки.


4. Для оценки качества модели мы будем ориентироваться на метрики качества Accuracy, precision, recall, F-мера. $Accuracy=\frac{TP+TN}{TP+TN+FN+FP}$ особого смысла рассматривать нет, так как она бесполезна в задачах с неравными классами. $ Precision = \frac{TP}{TP+FP}-$ точность(обладает сопсобностью отличать нек. класс от других классов). $ Recall = \frac{TP}{TP+FN}-$ полнота(обнаруживает класс вообще). $F-мера-$среднее гармоническое между precision и recall. Их анализ позволит нам оценить качество модели. Также для оценки точности классификации можно использовать $ROC-AUC$ кривую. Чтобы оценить нашу модель на **новых данных** можно использовать кросс-валидацию, для оценки будем брать абонентов, для которых уже известна целевая переменная. Важно брать абонентов из прошлого так, чтобы выборка на которой делается валидация была максимально похожа на ту ситуацию, которую модель должна предсказывать в настоящем.


5. Зная значение целевой переменной для текущей базы абонентов, можно как-то мотивировать абонентов с целевой переменной 1 на совершение покупки (акции и т.д). Вообщем предсказав, что клиент не совершит покупку больше года, мы как-то пытаемся повлиять на этого клиента, чтобы не было оттока. Как повлиять, это уже отдельный вопрос. Обычно используют A/B-тестирование для того, чтобы ввести новую фичу на интернет-ресурсе без возможности потери клиентов. (по поведению группы A принимается или отвергается некоторая статистическая гиотеза)